In [94]:
import cv2
import pandas as pd 
import numpy as np 
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [95]:
## image path 

imgdir = "data/LARD/LARD_test_real/LARD_test_real_edge_cases" # 原图文件夹
crops_dir = "images_cropped"  # ROI文件夹
csv_path = "data/LARD/LARD_test_real/LARD_test_real_edge_cases/Test_Real_Edge_Cases.csv"  # csv文件路径


In [96]:
## 根据真值裁剪ROI
def crop_and_save(image_dir, image_name, save_dir, x_A, y_A, x_B, y_B, x_C, y_C, x_D, y_D, show_points=False):
    # 读取原始图片
    img = Image.open(image_dir + '/' + image_name)
    
    ## 计算上下左右界
    lb = min(x_A, x_B, x_C, x_D)    # left bound
    rb = max(x_A, x_B, x_C, x_D)    # right bound
    tb = min(y_A, y_B, y_C, y_D)    # top bound 
    bb = max(y_A, y_B, y_C, y_D)    # bottom bound 
    
    # 计算原始区域的宽度和高度
    original_width = rb - lb 
    original_height = bb - tb 
    
    # 计算扩大后的宽度和高度
    # new_width = original_width * 2
    # new_height = original_height * 2
    
    ## 计算新的上下左右界
    new_lb = lb - (original_width // 2) if lb - (original_width // 2) > 0 else 0  # left bound
    new_rb = rb + (original_width // 2) if rb + (original_width // 2) < img.width else img.width  # right bound
    new_tb = tb - (original_height // 2) if tb - (original_height // 2) > 0 else 0  # top bound
    new_bb = bb + (original_height // 2) if bb + (original_height // 2) < img.height else img.height  # bottom bound
    
    # 计算截取区域的新坐标
    new_x_A = x_A - new_lb
    new_y_A = y_A - new_tb
    new_x_B = x_B - new_lb
    new_y_B = y_B - new_tb
    new_x_C = x_C - new_lb
    new_y_C = y_C - new_tb
    new_x_D = x_D - new_lb
    new_y_D = y_D - new_tb
    
    # 截取区域并保存为新的图片
    # (left, upper, right, lower)-tuple.
    cropped_img = img.crop((new_lb, new_tb, new_rb, new_bb))
    # plt.imshow(cropped_img)
    cropped_img.save(save_dir + '/' + image_name)
    if show_points:
        # 四个点的坐标
        points = {  'A': (new_x_A, new_y_A), 
                    'B': (new_x_B, new_y_B), 
                    'C': (new_x_C, new_y_C), 
                    'D': (new_x_D, new_y_D)}

        plt.figure(figsize=(10,10))
        # 绘制图片
        plt.imshow(cropped_img)

        # 绘制四个点
        for label, point in points.items():
            plt.scatter(point[0], point[1], color='red', )
            plt.text(point[0], point[1], label, fontsize=12, color='red')

        # 显示标记点
        plt.show()
    
    return new_x_A, new_y_A, new_x_B, new_y_B, new_x_C, new_y_C, new_x_D, new_y_D

In [97]:
# 读取csv 
df = pd.read_csv(csv_path, sep=';')
df.head()

,image,height,width,type,original_dataset,scenario,airport,runway,time_to_landing,weather,...,roll,watermark_height,x_A,y_A,x_B,y_B,x_C,y_C,x_D,y_D
0,images/-PJP3pgLgjI_022.png,2160,3840,real,REAL_Edge_Cases,NaN,EBBR,25L,30,rain,...,NaN,NaN,2038,1276,2055,1277,2017,1352,2069,1355
1,images/-PJP3pgLgjI_025.png,2160,3840,real,REAL_Edge_Cases,NaN,EBBR,25L,24,rain,...,NaN,NaN,1771,1327,1788,1327,1744,1413,1812,1413
2,images/-PJP3pgLgjI_027.png,2160,3840,real,REAL_Edge_Cases,NaN,EBBR,25L,20,rain,...,NaN,NaN,2018,1374,2037,1375,1986,1472,2069,1474
3,images/-PJP3pgLgjI_029.png,2160,3840,real,REAL_Edge_Cases,NaN,EBBR,25L,16,rain,...,NaN,NaN,1841,1319,1862,1319,1789,1438,1900,1442
4,images/-PJP3pgLgjI_030.png,2160,3840,real,REAL_Edge_Cases,NaN,EBBR,25L,14,rain,...,NaN,NaN,2016,1327,2037,1328,1952,1456,2089,1464


In [99]:
# 遍历 DataFrame 的每一行，裁剪并存储
crops_frame = {'image': [], 'x_A': [], 'y_A': [], 'x_B': [], 'y_B': [], 'x_C': [], 'y_C': [], 'x_D': [], 'y_D': []}
crops_df = pd.DataFrame(crops_frame)
for index, row in df.iterrows():
    # 使用列名称索引获取需要的元素
    img_path = row['image'] 
    x_A = row['x_A']
    y_A = row['y_A']
    x_B = row['x_B']
    y_B = row['y_B']
    x_C = row['x_C']
    y_C = row['y_C']
    x_D = row['x_D']
    y_D = row['y_D']
    
    ## 裁剪区域，获得新的四点坐标
    new_x_A, new_y_A, new_x_B, new_y_B, new_x_C, new_y_C, new_x_D, new_y_D = \
        crop_and_save(imgdir, img_path, crops_dir,x_A, y_A, x_B, y_B, x_C, y_C, x_D, y_D)
    ## 把新的四点坐标存起来
    crops_df.loc[index] = [img_path, new_x_A, new_y_A, new_x_B, new_y_B, new_x_C, new_y_C, new_x_D, new_y_D]

## 存储crops_df
crops_df.to_csv(crops_dir+'/'+'points.csv', index=False)

In [100]:
## 读取crops的信息并且计算左右的slope
slope_frame = {'image': [], 'x_A': [], 'y_A': [], 'x_B': [], 'y_B': [], 'x_C': [], 'y_C': [], 'x_D': [], 'y_D': [], 'slope_L': [], 'slope_R': []}
slope_df = pd.DataFrame(slope_frame)
for index, row in crops_df.iterrows():
    img_path = row['image'] 
    x_A = row['x_A']
    y_A = row['y_A']
    x_B = row['x_B']
    y_B = row['y_B']
    x_C = row['x_C']
    y_C = row['y_C']
    x_D = row['x_D']
    y_D = row['y_D']
    try:
        if x_A < x_B:
            if x_C < x_D:
                slope_L = (y_C - y_A) / (x_C - x_A)
                slope_R = (y_D - y_B) / (x_D - x_B)
            else:
                slope_L = (y_D - y_A) / (x_D - x_A)
                slope_R = (y_C - y_B) / (x_C - x_B)
        else:
            if x_C < x_D:
                slope_L = (y_C - y_B) / (x_C - x_B)
                slope_R = (y_D - y_A) / (x_D - x_A)
            else:
                slope_L = (y_D - y_B) / (x_D - x_B)
                slope_R = (y_C - y_A) / (x_C - x_A)
    except ZeroDivisionError:   # slope是0就跳过
        continue
    slope_df.loc[len(slope_df)] = [img_path, x_A, y_A, x_B, y_B, x_C, y_C, x_D, y_D, slope_L, slope_R]
slope_df.to_csv(crops_dir + '/' + 'slope.csv', index=False)

In [101]:
crops_df.head()

,image,x_A,y_A,x_B,y_B,x_C,y_C,x_D,y_D
0,images/-PJP3pgLgjI_022.png,47.0,39.0,64.0,40.0,26.0,115.0,78.0,118.0
1,images/-PJP3pgLgjI_025.png,61.0,43.0,78.0,43.0,34.0,129.0,102.0,129.0
2,images/-PJP3pgLgjI_027.png,73.0,50.0,92.0,51.0,41.0,148.0,124.0,150.0
3,images/-PJP3pgLgjI_029.png,107.0,61.0,128.0,61.0,55.0,180.0,166.0,184.0
4,images/-PJP3pgLgjI_030.png,132.0,68.0,153.0,69.0,68.0,197.0,205.0,205.0


In [102]:
slope_df.head()

,image,x_A,y_A,x_B,y_B,x_C,y_C,x_D,y_D,slope_L,slope_R
0,images/-PJP3pgLgjI_022.png,47.0,39.0,64.0,40.0,26.0,115.0,78.0,118.0,-3.619048,5.571429
1,images/-PJP3pgLgjI_025.png,61.0,43.0,78.0,43.0,34.0,129.0,102.0,129.0,-3.185185,3.583333
2,images/-PJP3pgLgjI_027.png,73.0,50.0,92.0,51.0,41.0,148.0,124.0,150.0,-3.062500,3.093750
3,images/-PJP3pgLgjI_029.png,107.0,61.0,128.0,61.0,55.0,180.0,166.0,184.0,-2.288462,3.236842
4,images/-PJP3pgLgjI_030.png,132.0,68.0,153.0,69.0,68.0,197.0,205.0,205.0,-2.015625,2.615385


In [7]:
df.loc[0]['image']

'images/BIRK_01_500_000.jpeg'

In [104]:
## 调用上交程序
line_df = pd.DataFrame({'image': [], 
                        'slope_L': [], 
                        'slope_R': []})
for index, row in slope_df.iterrows():
    img_path = row['image'] 
    slope_L, slope_R = detect_line(img_path, crops_dir) 
    line_df.loc[len(line_df)] = [img_path, slope_L, slope_R]
    
line_df.to_csv(crops_dir + '/' + 'line_slopes.csv', index=False)

C:\Users\wangs\AppData\Local\Temp/ipykernel_5048/606540378.py:47: RuntimeWarning: divide by zero encountered in float_scalars
  k_inv =  (x1 - x2) / (y1 - y2)
C:\Users\wangs\AppData\Local\Temp/ipykernel_5048/606540378.py:29: RuntimeWarning: invalid value encountered in double_scalars
  theta = np.sum(thetas * ds)/np.sum(ds)
C:\Users\wangs\AppData\Local\Temp/ipykernel_5048/606540378.py:30: RuntimeWarning: invalid value encountered in double_scalars
  rho = np.sum(rhos * ds)/np.sum(ds)


In [91]:
def detect_line(img_path, crops_dir):
    img = cv2.imread(crops_dir + '/'+ img_path) 
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # gray = hsv[:,:,0]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lsd = cv2.createLineSegmentDetector(0)
    lines, width, prec, nfa = lsd.detect(gray)

    # plt.figure(figsize=[10, 10])
    # plt.subplot(2, 2, 1)
    # plt.imshow(gray, 'gray')

    drawn_img = img.copy()
    for line in lines:
        x1, y1, x2, y2 = map(int, line[0])
        drawn_img = cv2.line(drawn_img, (x1, y1), (x2, y2), (0, 255, 0), 2, cv2.LINE_AA)
    # plt.subplot(2, 2, 2)
    # plt.imshow(cv2.cvtColor(drawn_img, cv2.COLOR_BGR2RGB))

    LLL = 2

    thetaList = []
    dList = []
    rhoList = []
    lineList = []
    for line in lines:
        line = line[0]
        theta, d, rho = computehoughTransformCoord(line[0], line[1], line[2], line[3])
        if d < LLL:
            continue
        if abs(np.abs(theta) - np.pi/2) < np.pi / 8 or np.abs(theta) > 1.5707963:
            continue
        thetaList.append(theta)
        dList.append(d)
        rhoList.append(rho)
        lineList.append(line)

    drawn_img = img.copy()
    for line in lineList:
        x1, y1, x2, y2 = map(int, line)
        if np.sqrt((x1 - x2)**2 + (y1 - y2)**2) < 0:
            continue
        drawn_img = cv2.line(drawn_img, (x1, y1), (x2, y2), (0, 255, 0), 2, cv2.LINE_AA)
    # plt.subplot(2, 2, 3)
    # plt.imshow(cv2.cvtColor(drawn_img, cv2.COLOR_BGR2RGB))

    N = 50
    Ntheta = N
    Nrho = N
    maxRho = np.sqrt(img.shape[0]**2 + img.shape[1]**2)
    # 创建一个网格
    theta_bins = np.linspace(-np.pi/2, np.pi/2, num=Ntheta)
    rho_bins = np.linspace(-maxRho, maxRho, num=Nrho)
    grid = np.zeros((len(theta_bins) - 1, len(rho_bins) - 1))
    TRgrid = [[[] for _ in range(len(rho_bins) - 1)] for _ in range(len(theta_bins) - 1)]

    # 映射thetaList和rhoList到网格中
    for theta, rho, d, line in zip(thetaList, rhoList, dList, lineList):
        theta_idx = np.digitize(theta, theta_bins) - 1
        rho_idx = np.digitize(rho, rho_bins) - 1
        # print(theta, rho, theta_idx, rho_idx)
        grid[theta_idx, rho_idx] += d
        TRgrid[theta_idx][rho_idx].append((theta, rho, line, d))

    # 绘制热力图
    # plt.subplot(2, 2, 4)
    # plt.imshow(grid, extent=(rho_bins[0], rho_bins[-1], theta_bins[0], theta_bins[-1]), aspect='auto', origin='lower')
    # plt.colorbar()
    # plt.xlabel('Rho')
    # plt.ylabel('Theta')
    # plt.show()

    # 找到最大元素的索引
    max_index = np.unravel_index(np.argmax(grid), grid.shape)
    max_d = grid[max_index]

    # 将最大元素设为负无穷
    grid[max_index] = -np.inf

    # 找到第二大元素的索引
    second_max_index = np.unravel_index(np.argmax(grid), grid.shape)
    second_max_d = grid[second_max_index]

    grid[second_max_index] = -np.inf

    third_max_index = np.unravel_index(np.argmax(grid), grid.shape)
    third_max_d = grid[third_max_index]

    # 输出最大元素和第二大元素的行索引和列索引
    # print("最大元素的行索引和列索引：", max_index)
    # print("第二大元素的行索引和列索引：", second_max_index)

    lineImg = img.copy()
    # slope_1, c1 = drawResult(TRgrid[max_index[0]][max_index[1]], lineImg, color = (0, 0, 255))
    # slope_2, c2 = drawResult(TRgrid[second_max_index[0]][second_max_index[1]], lineImg, color = (0, 255, 0))
    slope_1, c1 = drawResult(TRgrid[max_index[0]][max_index[1]])
    slope_2, c2 = drawResult(TRgrid[second_max_index[0]][second_max_index[1]])
    
    h = lineImg.shape[0]
    ## 求 y = h 时 x的值，小的是在左边
    try:
        x1b = int((h - c1) / slope_1)
        x2b = int((h - c2) / slope_2)
        ## 求 y = 0 时 x 的值
        x1t = int(-c1 / slope_1)
        x2t = int(-c2 / slope_2)
        if x1b < x2b:
            slope_L = slope_1
            slope_R = slope_2
            ## 左边红色，右边绿色
            cv2.line(lineImg, (x1t, 0), (x1b, h), (255, 0, 0), 2)
            cv2.line(lineImg, (x2t, 0), (x2b, h), (0, 255, 0), 2)
        else:
            slope_L = slope_2
            slope_R = slope_1
            ## 左边红色，右边绿色
            cv2.line(lineImg, (x2t, 0), (x2b, h), (255, 0, 0), 2)
            cv2.line(lineImg, (x1t, 0), (x1b, h), (0, 255, 0), 2)
            
        # cv2.line(lineImg, (0, int(c)), (LL, int( LL *m + c)), color, 2)
        # plt.figure()
        save_path = f"{crops_dir}/line_results/{img_path}"
        cv2.imwrite(save_path, cv2.cvtColor(lineImg, cv2.COLOR_BGR2RGB))
        # print(save_path)
        # plt.imshow(cv2.cvtColor(lineImg, cv2.COLOR_BGR2RGB))
    except:
        return None, None
    return slope_L, slope_R

In [84]:
# def drawResult(grid: list, lineImg, color = (0, 0, 255)):
def drawResult(grid: list):
    # 假设 max_index 是一个包含多个元组 (theta, rho) 的列表
    thetas = [pair[0] for pair in grid]
    rhos = [pair[1] for pair in grid]
    lines = [pair[2] for pair in grid]
    ds = [pair[3] for pair in grid]

    # x = []
    # y = []
    # weights = []
    # for line, d in zip(lines, ds):
    #     x.append(line[0])
    #     x.append(line[2])
    #     y.append(line[1])
    #     y.append(line[3])
    #     weights.append(d)
    #     weights.append(d)

    # A = np.vstack([x, np.ones(len(x))]).T
    # w = np.diag(np.sqrt(weights))
    # m, c = np.linalg.lstsq(w @ A, y @ w, rcond=None)[0]
    # print(m , c)

    thetas = np.asarray(thetas)
    rhos = np.asarray(rhos)
    ds = np.asarray(ds)
    
    theta = np.sum(thetas * ds)/np.sum(ds)
    rho = np.sum(rhos * ds)/np.sum(ds)

    m = 1/np.tan(-theta)
    c = rho / np.sin(theta)
    
    # # 在图像上绘制直线
    # LL = 3000
    # try:
    #     cv2.line(lineImg, (0, int(c)), (LL, int( LL *m + c)), color, 2)
    #     for line, d in zip(lines, ds):
    #         cv2.line(lineImg, (int(line[0]), int(line[1])), (int(line[2]), int(line[3])), (255, 0, 255), 1)
    # # print("line", max_d)
    # except:
    #     return None
    return m, c 

def computehoughTransformCoord(x1, y1, x2, y2):
    k_inv =  (x1 - x2) / (y1 - y2)
    theta = np.arctan(-k_inv)
    X0 = - k_inv * y1 + x1

    A = y2 - y1
    B = x2 - x1
    C = abs(x2*y1 - x1*y2)
    length = np.sqrt(A**2 + B**2)
    rho = C / length
    if X0 < 0:
        rho = -rho
    
    return theta, length, rho

